In [1]:
#install with pip
from pandas import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen

# built-in libraries
import datetime
import re
import sqlite3
import urllib
import csv
import datetime

In [2]:
# Read in state summary report for District, filter for needed fields and assign headers
insp = pd.read_csv("5fdinspi.csv", 
                   usecols=[2, 4, 5, 6, 7, 8, 9, 12, 13, 14, 17, 18, 80, 81], encoding="ISO-8859-1")
# ftp://dbprftp.state.fl.us/pub/llweb/5fdinspi.csv

insp.columns = ["county", "licnum", "sitename", "streetaddy", "cityaddy", "zip",
                "inspnum", "insptype", "inspdispos", "inspdate", "totalvio", "highvio", "licid", "visitid"]

# county_sought = 'Marion' # uncomment if particular county sought
# insp = insp[(insp.county == county_sought)] # uncomment if particular county sought
insp.sitename = insp.sitename.str.title() #titlecase sitename
insp.streetaddy = insp.streetaddy.str.title() #titlecase street adress
insp.cityaddy = insp.cityaddy.str.title() #titlecase city
insp = insp.applymap(lambda x: str(x).strip() if len(str(x).strip()) else None) #strip whitespace, replace empty vals

# takes LicenseID and VisitID, passes it into the urls for detailed reports later
result = []
for index, rows in insp.iterrows():
    visitid = rows['visitid']
    licid = rows['licid']
    urls = "https://www.myfloridalicense.com/inspectionDetail.asp? \
        InspVisitID= %s &licid= %s" % (visitid, licid)
    urls = urls.replace(' ', '')
    result.append(urls)
urlList = result

In [ ]:
# interate through the df to return tuples
# from collections import namedtuple
# var = list(insp.itertuples(index='visitid', name=None))

In [ ]:
# use tuple to put inspection summary data into database
# conn = sqlite3.connect('rinspecti.sqlite')
# c = conn.cursor()
# c.executemany('''INSERT OR IGNORE INTO fdinspi (county, licnum, sitename,
              streetaddy, cityaddy, zip, inspnum, insptype, inspdispos,
              inspdate, totalvio, highvio, licid, visitid, timestamp)
              VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', var)
# conn.commit()
# conn.close()

In [6]:
# iterate through the df to return library
insp_dict = [insp.to_dict(orient='index')]
len(insp_dict)

1

In [ ]:
# function to pull relevant info from scrape into a dictionary
def make_details_dict(url):
    inspurl = url
    inspectNum = inspurl[inspurl.rfind("VisitID=") + 1:8]
    
    html = urlopen(url)
    soup = bs4.BeautifulSoup(html.read(), 'lxml')
    details = soup.find_all('font', {'face': 'verdana'})[10:]
    result = []
    global detailsDict
  
    for detail in details:
        siteName = details[0].text
        licNum = details[2].text
        inspDate = details[6].text
        inspectDate = details[20].text
        numHigh = details[24].text
        numIntermed = details[25].text
        numBasic = details[26].text
        #if not(details[32].get_text):
        observed1 = details[32].get_text
        #elif not (details[34].text):
        #observed2 = details[34].get_text
        #observed3 = details[36].text
        #observed4 = details[38].text
        #observed5 = details[40].text
        #observed6 = details[42].text
        #observed7 = details[44].text
        #observed8 = details[46].text
        #observed9 = details[48].text
        #observed10 = details[50].text
        #observed11 = details[52].text
        #observed12 = details[54].text
        totalvioNum = int(numHigh) + int(numIntermed) + int(numBasic)

    detailsDict = {
        'inspectNum' : inspectNum,
        'restaurant': siteName,
        'license': licNum,
        'address' : siteAddress,
        'date': inspDate,
        'highvio' : numHigh,
        'totalvio' : totalvioNum,
        'observed1': observed1,
        #'observed2': observed2,
        #'observed3': observed3,
        #'observed4': observed4,
        #'observed5': observed5,
        #'observed6': observed6,
        #'observed7': observed7,
        #'observed8': observed8,
        #'observed9': observed9,
        #'observed10': observed10,
        #'observed11' : observed11,
        #'observed12' : observed12,
        }
    result.append(detailsDict)    
        
    return detailsDict

# iterate through urls calling function above
for url in urlList:
    make_details_dict(url)